<a href="https://colab.research.google.com/github/karthiksagarN/DeepFake-Detection/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Drive Mounting and dependencies installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=d849ad9b75aa0aa9291df080573db0498826dd9ef9702aa91909fffeaf1fd07b
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


### Importing necessary libararies

In [ ]:
import json
import glob
import numpy as np
import cv2
import copy
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

### Finding Frames

In [ ]:
# finding the number of frames for each video in dataset.
video_files = glob.glob('/content/drive/MyDrive/DeepFake/FF++/real/*.mp4')
video_files1 = glob.glob('/content/drive/MyDrive/DeepFake/FF++/fake/*.mp4')
video_files += video_files1

frame_count = []
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    if (int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150):
        video_files.remove(video_file)
        continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

print("Frames : ", frame_count)
print("Total Number of Videos : ", len(frame_count))
print("Average frames per video : ", np.mean(frame_count))

Frames :  [744, 955, 759, 322, 1000, 845, 840, 577, 1025, 590, 718, 845, 632, 1620, 1034, 835, 677, 915, 800, 568, 758, 642, 1415, 437, 405, 1111, 940, 356, 875, 461, 915, 932, 820, 447, 1185, 875, 755, 1193, 456, 383, 578, 718, 698, 796, 840, 724, 795, 1373, 518, 553, 487, 751, 703, 1100, 341, 900, 1015, 875, 847, 419, 830, 995, 1255, 786, 694, 297, 810, 1094, 999, 678, 890, 819, 803, 1080, 860, 975, 790, 1325, 885, 1380, 1023, 723, 1015, 820, 880, 960, 529, 1045, 733, 743, 960, 729, 865, 1335, 558, 925, 1620, 384, 711, 878, 748, 657, 855, 1050, 1110, 869, 1350, 629, 932, 815, 159, 952, 1430, 518, 771, 970, 955, 1391, 706, 892, 870, 1457, 880, 549, 1020, 609, 880, 1350, 539, 689, 1620, 1330, 926, 775, 890, 429, 845, 1326, 655, 1384, 865, 800, 1001, 1012, 880, 1465, 924, 572, 672, 722, 1002, 664, 713, 839, 446, 208, 458, 446, 1453, 1082, 990, 685, 974, 1269, 815, 650, 960, 574, 446, 955, 910, 940, 301, 940, 816, 395, 534, 563, 481, 533, 655, 861, 684, 533, 543, 875, 456, 835, 932, 471,

### EXTRACTING FACE CROPPED IMAGES FROM VIDEOS

In [ ]:
import cv2
import os
import face_recognition

# Define the paths to the folders containing the videos
videos_folder_1 = "/content/drive/MyDrive/DeepFake/FF++/fake"
videos_folder_2 = "/content/drive/MyDrive/DeepFake/FF++/real"

# Define the path to the output folder for face-only videos
output_folder1 = "/content/drive/MyDrive/DeepFake/FF++/facecropped/fake"
output_folder2 = "/content/drive/MyDrive/DeepFake/FF++/facecropped/real"

# Function to extract faces from a video and save them as images
def extract_faces_from_video(video_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    frames_processed = 0

    # Read until video is completed
    while frame_count < 5:
        # Read a single frame
        ret, frame = video_capture.read()
        if not ret:
            break  # Break the loop if no more frames are available

        # Convert frame to RGB (face_recognition library requires RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find faces in the frame
        face_locations = face_recognition.face_locations(rgb_frame)

        # If faces are found, extract and save them
        for top, right, bottom, left in face_locations:
            # Extract the face region
            face_image = frame[top:bottom, left:right]

            # Write the face image to the output folder
            output_path = os.path.join(output_folder, f"{os.path.basename(video_path)}_frame_{frame_count}.jpg")
            cv2.imwrite(output_path, face_image)

            # Increment frame count
            frame_count += 1
            frames_processed += 1

        if frames_processed == 5:
            break

    # Release the video capture object
    video_capture.release()

# Process videos in the first folder
for video_file in os.listdir(videos_folder_1):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(videos_folder_1, video_file)
        extract_faces_from_video(video_path, output_folder1)

# Process videos in the second folder
for video_file in os.listdir(videos_folder_2):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(videos_folder_2, video_file)
        extract_faces_from_video(video_path, output_folder2)

### EXTRACTING FACE CROPPED VIDEOS

In [ ]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm

# Function to extract frames from a video
def extract_frames(video_path):
    frames = []
    vidObj = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Function to detect faces in frames and save cropped face video clips
def create_face_videos(video_paths, output_dir):
    already_present_count = len(glob.glob(os.path.join(output_dir, '*.mp4')))
    print("Number of videos already present: ", already_present_count)

    for video_path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(video_path))
        if os.path.exists(out_path):
            print("File already exists: ", out_path)
            continue

        frames = extract_frames(video_path)
        frame_count = len(frames)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))

        for idx, frame in enumerate(frames):
            if idx <= 150:  # Process only the first 150 frames
                faces = face_recognition.face_locations(frame)
                if faces:
                    top, right, bottom, left = faces[0]
                    face_frame = frame[top:bottom, left:right, :]
                    resized_face = cv2.resize(face_frame, (112, 112))
                    out.write(resized_face)

        out.release()

# Example usage
video_paths = glob.glob("/content/drive/MyDrive/DeepFake/FF++/real/*.mp4")
output_directory = '/content/drive/MyDrive/DeepFake/FF++/facecropped/real'
create_face_videos(video_paths, output_directory)


Number of videos already present:  0


100%|██████████| 150/150 [7:12:33<00:00, 173.02s/it]


In [ ]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm

# Function to extract frames from a video
def extract_frames(video_path):
    frames = []
    vidObj = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Function to detect faces in frames and save cropped face video clips
def create_face_videos(video_paths, output_dir):
    already_present_count = len(glob.glob(os.path.join(output_dir, '*.mp4')))
    print("Number of videos already present: ", already_present_count)

    for video_path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(video_path))
        if os.path.exists(out_path):
            print("File already exists: ", out_path)
            continue

        frames = extract_frames(video_path)
        frame_count = len(frames)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))

        for idx, frame in enumerate(frames):
            if idx <= 150:  # Process only the first 150 frames
                faces = face_recognition.face_locations(frame)
                if faces:
                    top, right, bottom, left = faces[0]
                    face_frame = frame[top:bottom, left:right, :]
                    resized_face = cv2.resize(face_frame, (112, 112))
                    out.write(resized_face)

        out.release()

# Example usage
video_paths = glob.glob("/content/drive/MyDrive/DeepFake/FF++/fake/*.mp4")
output_directory = '/content/drive/MyDrive/DeepFake/FF++/facecropped/fake'
create_face_videos(video_paths, output_directory)


Number of videos already present:  72


  0%|          | 0/150 [00:00<?, ?it/s]

File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/06_27__walking_and_outside_surprised__O7L5Z9U8.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/03_07__walking_outside_cafe_disgusted__IFSURI9X.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/03_04__outside_talking_pan_laughing__T04P6ELC.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/03_06__podium_speech_happy__83ABVHC3.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/03_07__walk_down_hall_angry__IFSURI9X.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/02_27__walk_down_hall_angry__78M8S6M6.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/02_18__walking_down_street_outside_angry__21JTDDEL.mp4
File already exists:  /content/drive/MyDrive/DeepFake/FF++/facecropped/fake/03_07__walk_down_hall_angry__6PHZRQ4H.mp4
File already exis

100%|██████████| 150/150 [3:48:37<00:00, 91.45s/it] 


In [ ]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm

# Function to extract frames from a video
def extract_frames(video_path):
    frames = []
    vidObj = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Function to detect faces in frames and save cropped face video clips
def create_face_videos(video_paths, output_dir):
    already_present_count = len(glob.glob(os.path.join(output_dir, '*.mp4')))
    print("Number of videos already present: ", already_present_count)

    for video_path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(video_path))
        if os.path.exists(out_path):
            print("File already exists: ", out_path)
            continue

        frames = extract_frames(video_path)
        frame_count = len(frames)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))

        for idx, frame in enumerate(frames):
            if idx <= 150:  # Process only the first 150 frames
                faces = face_recognition.face_locations(frame)
                if faces:
                    top, right, bottom, left = faces[0]
                    face_frame = frame[top:bottom, left:right, :]
                    resized_face = cv2.resize(face_frame, (112, 112))
                    out.write(resized_face)

        out.release()

# Example usage
video_paths = glob.glob("/content/drive/MyDrive/DeepFake/CELEB-DF/real/*.mp4")
output_directory = '/content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real'
create_face_videos(video_paths, output_directory)


Number of videos already present:  148


  0%|          | 0/405 [00:00<?, ?it/s]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video796.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video797.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video798.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video799.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video800.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video801.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video802.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video803.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video804.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video805.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecro

 12%|█▏        | 50/405 [00:42<04:58,  1.19it/s]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video848.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video850.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video851.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video852.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video853.mp4


...

 99%|█████████▊| 399/405 [2:43:14<03:26, 34.37s/it]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video1054.mp4


 99%|█████████▉| 402/405 [2:44:35<01:33, 31.15s/it]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/real/Video1108.mp4


100%|██████████| 405/405 [2:45:55<00:00, 24.58s/it]


In [ ]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm

# Function to extract frames from a video
def extract_frames(video_path):
    frames = []
    vidObj = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Function to detect faces in frames and save cropped face video clips
def create_face_videos(video_paths, output_dir):
    already_present_count = len(glob.glob(os.path.join(output_dir, '*.mp4')))
    print("Number of videos already present: ", already_present_count)

    for video_path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(video_path))
        if os.path.exists(out_path):
            print("File already exists: ", out_path)
            continue

        frames = extract_frames(video_path)
        frame_count = len(frames)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))

        for idx, frame in enumerate(frames):
            if idx <= 150:  # Process only the first 150 frames
                faces = face_recognition.face_locations(frame)
                if faces:
                    top, right, bottom, left = faces[0]
                    face_frame = frame[top:bottom, left:right, :]
                    resized_face = cv2.resize(face_frame, (112, 112))
                    out.write(resized_face)

        out.release()

# Example usage
video_paths = glob.glob("/content/drive/MyDrive/DeepFake/CELEB-DF/fake/*.mp4")
output_directory = '/content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake'
create_face_videos(video_paths, output_directory)


Number of videos already present:  148


  0%|          | 0/795 [00:00<?, ?it/s]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video252.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video52.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video580.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video581.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video53.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video80.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video81.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video54.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video271.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video82.mp4
File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/f

  6%|▌         | 49/795 [00:38<09:47,  1.27it/s]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video195.mp4


...

 41%|████▏     | 329/795 [2:21:05<3:12:33, 24.79s/it]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video118.mp4


 42%|████▏     | 334/795 [2:23:07<3:07:44, 24.43s/it]

File already exists:  /content/drive/MyDrive/DeepFake/CELEB-DF/facecropped/fake/Video119.mp4


 43%|████▎     | 338/795 [2:24:53<3:28:41, 27.40s/it]